**Общая информация**

**Срок сдачи:** 29 марта 2021, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [2]:
from scipy.spatial.distance import cdist
from scipy import stats
from sklearn.neighbors import KDTree

In [3]:
class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute', metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
    
    def fit(self, X, y):
        self.X = X
        self.Y = y
        self.tree = KDTree(self.X, leaf_size=30, metric='euclidean')
    
    def predict(self, X):
        if self.algorithm == 'brute':
            dist = cdist(X, self.X, metric='euclidean')
            neighbors = np.argpartition(dist, self.n_neighbors, axis=1)[:,:self.n_neighbors]
            return stats.mode(self.Y[neighbors], axis=1).mode.ravel()
        elif self.algorithm == 'kd_tree':
            _, neighbors = self.tree.query(X, k=self.n_neighbors)
            return stats.mode(self.Y[neighbors], axis=1).mode.ravel()
            
    

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [4]:
iris = datasets.load_iris()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [6]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [7]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [8]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [9]:
%time clf.fit(X_train, y_train)

CPU times: user 4.37 ms, sys: 1.08 ms, total: 5.45 ms
Wall time: 2.85 ms


KNeighborsClassifier(algorithm='brute', n_neighbors=2)

In [10]:
%time my_clf.fit(X_train, y_train)

CPU times: user 590 µs, sys: 145 µs, total: 735 µs
Wall time: 580 µs


In [11]:
%time clf.predict(X_test)

CPU times: user 0 ns, sys: 10.5 ms, total: 10.5 ms
Wall time: 6.11 ms


array([2, 1, 1, 0, 2, 2, 0, 2, 0, 0, 2, 0, 1, 2, 1])

In [12]:
%time my_clf.predict(X_test)

CPU times: user 11.2 ms, sys: 40 µs, total: 11.2 ms
Wall time: 6.42 ms


array([2, 1, 1, 0, 2, 2, 0, 2, 0, 0, 2, 0, 1, 2, 1])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [13]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [14]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [15]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 4.87 ms, total: 4.87 ms
Wall time: 2.95 ms


KNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)

In [16]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 652 µs, total: 652 µs
Wall time: 506 µs


In [17]:
%time clf.predict(X_test)

CPU times: user 3.67 ms, sys: 12 ms, total: 15.7 ms
Wall time: 9.31 ms


array([1, 2, 1, 1, 1, 0, 2, 1, 0, 0, 2, 1, 0, 2, 0])

In [18]:
%time my_clf.predict(X_test)

CPU times: user 10.8 ms, sys: 2.74 ms, total: 13.5 ms
Wall time: 7.81 ms


array([1, 2, 1, 1, 1, 0, 2, 1, 0, 0, 2, 1, 0, 2, 0])

In [19]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [20]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [21]:
data = np.asarray(newsgroups['data'])
target = np.asarray(newsgroups['target'])

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
vectorizer = CountVectorizer(stop_words='english')
counts = vectorizer.fit_transform(data)

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [24]:
from sklearn.model_selection import KFold

In [25]:
def get_split(data, target, n_splits=3):
    folds = KFold(n_splits)

    for train_index, test_index in folds.split(data):
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = target[train_index], data[test_index]
        yield X_train, X_test, y_train, y_test

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [26]:
def average_accuracy(knn_classifier, data, target, n_splits=3):
    result = 0
    for X_train, X_test, y_train, y_test in get_split(data, target, n_splits):
        knn_classifier.fit(X_train, y_train)
        pred = knn_classifier.predict(X_test)
        result += accuracy_score(y_test, my_clf_pred)
    return result / n_splits

In [27]:
clf = KNeighborsClassifier(n_neighbors=2)
average_accuracy(clf, data, target)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/fourthrome/miniconda3/envs/sphere-py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-a05813d513bf>", line 2, in <module>
    average_accuracy(clf, data, target)
  File "<ipython-input-26-19793d42c701>", line 4, in average_accuracy
    knn_classifier.fit(X_train, y_train)
  File "/home/fourthrome/miniconda3/envs/sphere-py38/lib/python3.8/site-packages/sklearn/neighbors/_classification.py", line 179, in fit
    return self._fit(X, y)
  File "/home/fourthrome/miniconda3/envs/sphere-py38/lib/python3.8/site-packages/sklearn/neighbors/_base.py", line 363, in _fit
    X, y = self._validate_data(X, y, accept_sparse="csr",
  File "/home/fourthrome/miniconda3/envs/sphere-py38/lib/python3.8/site-packages/sklearn/base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/home/fourthrome/miniconda3/env

TypeError: object of type 'NoneType' has no len()

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.